In [1]:
import numpy as np
import matplotlib.pyplot as plt

import gym
from gym.spaces import Discrete, Box

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import Callback
from spinup import ppo
import tensorflow as tf

Using TensorFlow backend.


# Reinforcement Learning com Jogo da Velha e TensorFlow

#### Por Bruna Kimura, Elisa Malzoni e Raphael Costa

## Intro

Essa aula tem como objetivo discorrer e mostrar dois métodos de Reinforcement Learning: o Proximal Policy Optimization (PPO) e o Deep Q-Learning.

Para tal, utilizaremos um ambiente simulado de um jogo da velha. No jogo da velha, dois jogadores duelam e, quem conseguir 3 Círculos (jogador 1) ou 3 X's (Jogador 2) em sequência em uma linha, coluna ou diagonal vence. Nosso objetivo nesta aula é construir um Bot que aprenda a jogar o jogo da velha, e, mais do que isso, ganhe o máximo possível, permitindo-se apenas empatar.

## Explicações Iniciais

![diagrama](img/rl.png)

Para um experimento de Reinforcement Learning, precisamos definir 4 conceitos principais: Agente, Estado, Ambiente e Recompensa.

O agente é aquele que realiza ações. No nosso caso, queremos programar um agente capaz de jogar o Jogo da Velha (e, é claro, vencer). Este Agente é treinado dentro de um ambiente, na qual o Agente se baseia para tomar suas decisões. No nosso caso, este ambiente será uma matriz de 9 posições, simulando o tabuleiro de jogo da velha. Cada passo que o Agente da dentro do ambiente chamamos de Ação, ou seja, são todas as possibilidades de ação que o Agente tem. Cada Ação leva o Agente para um novo Estado, que corresponde a uma situação concreta e imediata da onde o Agente se encontra. Ainda, cada vez que o Agente faz uma Ação e se encontra em um novo Estado, definimos uma Recompensa dada por aquela Ação. No nosso jogo, a maior recompensa que o Agente terá é 1, dada quando ele consegue uma vitória. Quando perder, sua recompensa será de -1, e, por último, a recompensa por um empate é 0 (Nula).

## O ambiente

![tabuleiro](img/tabuleiro.png)

Para construir o ambiente, utilizaremos duas classes: TicTacToe e TicTacToeEnv.

Na <code>TicTacToe</code> definiremos o que é o jogo da velha, é a classe que descreve as regras do jogo em si. Assim, ela simulará uma matriz de 9 posições, que podem assumir 3 diferentes valores: 0, 1 e 2, onde 0 é uma posição vazia, 1 é o jogador círculo e 2 é o jogador X. Ainda, é ela quem confere se há um ganhador após determinada jogada.

Ja a <code>TicTacToeEnv</code> é a classe responsável por efetivamente ser o ambiente de Reinforcement Learning do nosso bot. Ela baseia-se na classe TicTacToe para efetuar movimentos, treinar o bot, etc.

Para codificarmos um ambiente de Reinforcement Learning precisamos basicamente de duas funções: reset e step.

A função <code>reset</code>, como o próprio nome diz, tem o papel de iniciar o ambiente a cada iteração. Assim, ela iniciará a matriz com todas as 9 posições zeradas e efetuará a primeira jogada do jogador 1. As jogadas do jogador 1, aquele contra o qual nosso bot está jogando, serão efetuadas aleatoriamente.

A função <code>step</code> é responsável por indicar, no ambiente, qual foi a jogada que o nosso bot realizou. Ainda, ela também é responsável por dizer se o jogo acabou. Portanto, seu retorno é o estado do tabuleiro após a jogada do bot, a recompensa do agente para aquela ação tomada e se o jogo terminou.  

Para este projeto, o design das funções do TicTacToeEnv foi baseado nos enviroments especificados na biblioteca do Keras, já que esta possui a implementação do algoritmo de Deep Q-Learning e, a biblioteca que possui a implementação do PPO, a OpenAI Spinning Up, é baseada na biblioteca do Keras para simular seus ambientes.

### TicTacToe

In [34]:
class TicTacToe:

    def __init__(self):
        self.board_state = None
    
    def set_state(self, new_state):
        """ 2d array of cell positions of the board. 0 = cell not occupied,
            1 = cross occupies cell, 2 = nought occupies cell.
            Example: [
                [0, 0, 1],
                [0, 0, 2],
                [0, 0, 0]
            ] """

        new_state = np.array(new_state)

        assert new_state.shape == (len(new_state), len(new_state))

        self.board_state = new_state

        return self.board_state

    def is_finished(self):
        """ 0 = not finished, 1 = cross win, 2 = nought win, 3 = tie """

        # Are we tied?
        if self.board_state.flatten().tolist().count(0) == 0:
            return 3

        # Stolen: https://codereview.stackexchange.com/a/24775
        positions_groups = (
            [[(x, y) for y in range(self.get_board_size())] for x in range(self.get_board_size())] +  # horizontals
            [[(x, y) for x in range(self.get_board_size())] for y in range(self.get_board_size())] +  # verticals
            [[(d, d) for d in range(self.get_board_size())]] +  # diagonal from top-left to bottom-right
            [[(2-d, d) for d in range(self.get_board_size())]]  # diagonal from top-right to bottom-left
        )
        for positions in positions_groups:
            values = [self.board_state[x][y] for (x, y) in positions]
            if len(set(values)) == 1 and values[0]:
                return values[0]

        # Game isn't finished
        return 0

    def get_board_size(self):
        return len(self.board_state)

    def get_turn(self):
        """ Returns 1 for crosses turn, 2 for noughts turn """

        flattened_list = self.board_state.flatten().tolist()

        if flattened_list.count(1) > flattened_list.count(2):
            return 2
        else:
            return 1

    def make_move(self, x, y):
        """ Updates the state with the requested new occupied cell """
        x = int(x)
        y = int(y)
        
        # Sanity check
        assert x < self.get_board_size() and y < self.get_board_size()
        assert self.board_state[y][x] == 0
        assert self.is_finished() == 0

        new_state = self.board_state.copy()
        new_state[y][x] = self.get_turn()

        return self.set_state(new_state)

    @staticmethod
    def translate_position_to_xy(position, board_size=3):
        """ Takes a single number and maps it to x, y coordinates.
            Example: 8 = 2, 2 for a board_size of 3 """

        x = position % board_size
        y = position / board_size

        return x, y### TicTacToe

### TicTacToeEnv

In [35]:
### TicTacToeclass TicTacToeEnv:
class TicTacToeEnv:
    action_space = Discrete(3**2)

    def __init__(self, board_size=3, predict_for=None):
        self.board_size = board_size
        self.predict_for = predict_for

        self.observation_space = Box(
            low=np.array([0 for cell in range(self.board_size ** 2)]),
            high=np.array([2 for cell in range(self.board_size ** 2)])
        )
        self.tictactoe = None
        
    def render(self):
        if self.tictactoe is not None and self.tictactoe.board_state is not None:
            print(self.tictactoe.board_state)
                    
    def reset(self):
        if self.predict_for is not None:
            self.tictactoe = TicTacToe()
            self.tictactoe.set_state(self.predict_for)
            return self.tictactoe.board_state.flatten()

        self.tictactoe = TicTacToe()
        self.tictactoe.set_state([
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ])
        move = self._get_random_move()

        self.tictactoe.make_move(move[0], move[1])

        return self.tictactoe.board_state.flatten()
    
    def resetX1(self):
        self.tictactoe = TicTacToe()
        self.tictactoe.set_state([
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ])

        return self.tictactoe.board_state.flatten()
    
    def stepX1(self, action):
        translated_action = TicTacToe.translate_position_to_xy(action)
        
        try:
            self.tictactoe.make_move(translated_action[0], translated_action[1])

        except AssertionError:
            return self.tictactoe.board_state.flatten(), -1, True, {}
        
        winner = self.tictactoe.is_finished()
        #winner == 0: nao acabou
        #winner == 1: ganhou o 1
        #winner == 2: ganhou o 2
        #winner == 3: empate
        return self.tictactoe.board_state.flatten(), winner 
    
        

    def step(self, action):
        if self.predict_for is not None:
            return self.tictactoe.board_state.flatten(), 0, True, {}

        translated_action = TicTacToe.translate_position_to_xy(action)
        

        try:
            self.tictactoe.make_move(translated_action[0], translated_action[1])

        except AssertionError:
            return self.tictactoe.board_state.flatten(), -1, True, {}

        reward = 0
        done = False
        winner = self.tictactoe.is_finished()
        if winner == 0:
            move = self._get_random_move()
            self.tictactoe.make_move(move[0], move[1])

            next_winner = self.tictactoe.is_finished()
            if next_winner == 1:
                reward = -1
                done = True
            elif next_winner == 3:
                reward = 0
                done = True

        elif winner == 2:
            reward = 1
            done = True

        elif winner == 3:
            reward = 0
            done = True

        return self.tictactoe.board_state.flatten(), reward, done, {}

    def _get_random_move(self):
        assert self.tictactoe.is_finished() == 0

        positions = []
        for x in range(self.board_size):
            for y in range(self.board_size):
                if self.tictactoe.board_state[y][x] == 0:
                    positions.append((x, y))

        return positions[np.random.choice(len(positions), 1)[0]]

### TicTacToe com o PPO


Para o Proximal Policy Optimization, como dito anteriormente, utilizaremos a implementação do OpenAI Spinning Up. Inicialmente, definimos uma váriavel que aponta para o ambiente que queremos utilizar, a <code>env_fn</code>.

Ainda, a biblioteca define um Logger, a váriavel <code>logger_kwargs</code> que é a classe que printa o andamento do treinamento (como pode ser visto ao rodar as células seguintes) e salva os pesos e histórico do experimento em arquivos. Neste caso, salvaremos tais dados na pasta que este Notebook se encontra. 

Por fim, chamamos efetivamente a função <code>ppo</code>, passando, basicamente, 5 argumentos obrigatórios:
- <code>env_fn</code>: váriavel do ambiente
- <code>ac_kwargs</code>: váriavel que define a rede neural que será utilizada no experimento.
- <code>steps_per_epoch</code>: Quantos passos por epóca o ambiente pode dar
- <code>epochs</code>: número de épocas que serão realizadas no treinamento
- <code>logger_kwargs</code>: váriavel do logger

In [36]:
env_fn = lambda : TicTacToeEnv()

### Atenção!!! O treinamento demora, não execute-a se você não deseja sobrepor o treinamento que ja fizemos.

In [ ]:
ac_kwargs = dict(hidden_sizes=[64,64], activation=tf.nn.relu)

logger_kwargs = dict(output_dir='spinupPpo', exp_name='experiment')

ppo(env_fn=env_fn, ac_kwargs=ac_kwargs, steps_per_epoch=5000, epochs=50, logger_kwargs=logger_kwargs)

### Pontos a se observar


No logger do treinamento feito acima, é importante se atentar a alguns valores de retorno. Para uma Epóca N, podemos analisar o <code>AverageEpRet</code>, que indica o valor médio de retorno dos episódios. Para a época 9, por exemplo, temos uma média de -0.152, sendo que o máximo de retorno é 1 (quando o bot ganha) e o mínimo é -1, que é quando o bot perde o jogo. Uma média de -0.152 indica que o bot ainda está perdendo mais vezes do que ganhando, o que nos leva a conclusão de que é necessário mais épocas de treinamento, já que, idealmente queremos, no máximo, empatar e, se possível, nunca perder.

Ainda, podemos analisar o EpLen, que indica uma média da duração do episódio, ou seja, quantas vezes o nosso bot jogou, em média, cada jogo. Considerando que ele joga em segundo, o bot pode jogar no máximo 4 vezes, preenchendo os 9 campos, resultando em um empate, e, no mínimo, 2 vezes, considerando que ele não evitou a primeira chance do outro player ganhar. Interessante analisar que o EpLen, no ínicio, vai crescendo junto com o AverageEpRet, o que indica que o bot aprende primeiro a evitar as derrotas.


Ao final das 50 épocas de treinamento, o AverageEpRet encontra-se em 0.714, o que corresponde a uma média boa e próxima de 1. 

### Testando nosso bot 

Para testar nosso bot treinado com o PPO, utilizamos a função <code>load_policy</code>, que carrega os pesos e váriaveis salvos no treinamento nos arquivos locais. Vamos carregar nosso modelo para obter a função <code>get_action</code>, que é a função que, ao passarmos um tabuleiro, retorna outro tabuleiro com a jogada que o bot tomou a partir daquele estado

In [37]:
from spinup.utils.test_policy import load_policy, run_policy
_, get_action = load_policy('./spinupPpo')
env = env_fn()
#run_policy(env, get_action, render=False)

INFO:tensorflow:Restoring parameters from ./spinupPpo/simple_save/variables/variables
Using default action op.


Apesar de podermos testar nosso bot com a função <code>run_policy</code> da biblioteca do OpenAI, iremos fazer nossa própria função de teste para fins didáticos.

Na função play, passaremos o argumento n, que representa apenas quantas jogos serão feitos. Para cada jogo, iniciamos chamando a função <code>reset</code> do ambiente, para iniciarmos a matriz e fazer a primeira jogada do jogador 1. 

Assim, iniciamos a váriavel d, que indica se o jogo terminou ou não. Enquanto o jogo não terminar, efetuamos jogadas com a função <code>get_action</code>. Após a <code>get_action</code>, indicamos ao ambiente a jogada feita pelo bot com a função <code>step</code>, que atualiza o estado do tabuleiro e verifica se há um ganhador, e, se não houver, realiza a jogada do jogador 1. Esta função retorna 3 principais valores: 
- <code>o</code>, que representa o estado do tabuleiro;
- <code>r</code>, que representa a recompensa do Agente para aquela ação tomada;
- <code>d</code>, que inidica se o jogo terminou;

In [38]:
def play(n=1):
    for i in range(n):
        o = env.reset()
        env.render()
        d = False
        while not d:
            a = get_action(o)
            o, r, d, _ = env.step(a)
            env.render()
        if(r == 0):
            print("Deu velha")
        elif(r == 1):
            print("Bot ganhou")
        elif(r == -1):
            print("Bot Perdeu")
        print("-------------------")
            

In [40]:
play(2)

[[0 0 0]
 [0 1 0]
 [0 0 0]]
[[0 0 0]
 [0 1 1]
 [0 0 2]]
[[0 0 1]
 [0 1 1]
 [2 0 2]]
[[0 0 1]
 [0 1 1]
 [2 2 2]]
Bot ganhou
-------------------
[[1 0 0]
 [0 0 0]
 [0 0 0]]
[[1 0 1]
 [0 0 0]
 [2 0 0]]
[[1 0 1]
 [0 0 0]
 [2 1 2]]
[[1 0 1]
 [0 2 1]
 [2 1 2]]
[[1 1 1]
 [2 2 1]
 [2 1 2]]
Deu velha
-------------------


### TicTacToe com Deep Q-Learning

Para a o Deep Q-Learning utilizaremos a biblioteca do keras-rl. Como dito anteriormente, nosso ambiente foi feito com base na biblioteca do keras, portanto, não devemos ter muito problema para utilizar sua implementação. 

A classe <code>ModelIntervalCheckpoint</code>, como o próprio nome sugere, é responsável por salvar certos pontos de checkpoint durante o treinamento e, também é ela quem salva localmente os arquivos com os pesos encontrados para a rede neural em cada episódio, o que também tinhamos na execução do PPO.

In [41]:
class ModelIntervalCheckpoint(Callback):
    def __init__(self, interval, verbose=0):
        super(ModelIntervalCheckpoint, self).__init__()
        self.interval = interval
        self.step = 0

        self.rewards = []
        self.last_max = -1

    def reset(self):
        self.rewards = []

    def on_step_begin(self, step, logs):
        if self.step % self.interval == 0:
            if len(self.rewards) > 0:
                mean_reward = np.nanmean(self.rewards, axis=0)
                if mean_reward > self.last_max:
                    filename = 'saved-weights/%s.h5f' % mean_reward
                    print("\nSaving model checkpoint with mean reward %s to %s" % (mean_reward, filename))

                    self.model.save_weights(filename, overwrite=True)

                    self.last_max = mean_reward

            self.reset()

    def on_step_end(self, step, logs={}):

        self.rewards.append(logs['reward'])
        self.step += 1

Novamente criamos uma váriavel <code>env</code> em referência ao nosso ambiente, o <code>TicTacToeEnv</code>. A váriavel <code>model</code> será responsável por indicar o formato da Rede Neural que utilizaremos durante o experimento. 

Um conceito, que não há no PPO, é o de guardar os estados que o Agente ja passou, evitando que ele esqueça totalmente onde ja passou em passos anteriores. Para isso, criamos a váriavel <code>memory</code>. Ainda, criamos a <code>policy</code>, que diz respeito a policy utilizada pelo Agente.

Por fim, criamos um objeto do da classe <code>DQNAgent</code>, que recebe 7 argumentos obrigatórios:
- <code>model</code>: modelo de rede neural que utilizaremos no experimento.
- <code>nb_actions</code>: número de ações possíveis para o Agente (no nosso caso, 9, pois existem 9 posições possíveis para ele jogar)
- <code>memory</code>: memória das jogadas anteriores
- <code>target_model_update</code>: hiperparâmetro do modelo, que inidica a frequência com que atualizaremos os valores na rede neural. Para casos em que a váriavel está entre 0 e 1, estamos em Soft Update, e para casos em que a váriavel é maior que 1, estamos em Hard Update. No soft update, atualizamos os valores da rede neural por partes, enquanto que no hard update atualizamos a rede neural como um todo a cada <code>target_model_update</code> passos.
- <code>policy</code>: policy utilizada pelo Agente.


Para efetivamente treinarmos o modelo, precisamos chamar 2 funções da biblioteca:

1) <code>Compile</code>: compila o Agente que será utilizado, determina um Optmizer para ser utilizado no treinamento e recebe a métrica sobre o qual queremos analisar o desempenho. No nosso caso, utilizaremos o mae, mean absolut error.

2) <code>fit</code>: Treina o Agente sobre o ambiente. 

In [42]:
def build_dqn(env):
    nb_actions = env.action_space.n

    model = Sequential()
    model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions, activation='linear'))

    memory = SequentialMemory(limit=5000000, window_length=1)
    policy = BoltzmannQPolicy()
    log_interval = 10000

    dqn = DQNAgent(
        model=model,
        nb_actions=nb_actions,
        memory=memory,
        target_model_update=1e-2,
        policy=policy
    )

    dqn.compile(Adam(lr=1e-5), metrics=['accuracy', 'mae'])

    return dqn

def predict(board_state, model_path):
    env = TicTacToeEnv(predict_for=board_state)

    dqn = build_dqn(env)

    dqn.load_weights(model_path)

    dqn.test(env, nb_episodes=1, visualize=False, verbose=0)

    return dqn.recent_action

In [43]:
env = TicTacToeEnv()

dqn = build_dqn(env)

### Atenção!!! O treinamento demora, não execute-a se você não deseja sobrepor o treinamento que ja fizemos.

In [12]:
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1,
    callbacks=[ModelIntervalCheckpoint(interval=log_interval)],
    log_interval=log_interval
)

Instructions for updating:
Colocations handled automatically by placer.


Para testar, basta escolher o arquivo de pesos gerado com a maior média de retorno, que corresponde ao <code>AverageEpRet</code> no ambito do PPO. Portanto, a principio queremos um valor maior que 0, que inidicaria, novamente, que o Agente ganhou mais partidas do que perdeu. Porém, quando colocamos este Agente para treinar com um nb_steps=5000000, conseguimos com que esta média fosse de -0.141, o que pode indicar que, utilizando PPO, conseguimos um melhor resultado mais rápido, ja que, para treinar o PPO, precisamos de apenas 50 épocas para chegar em uma média de 0.751.

In [44]:
def playDql(n=1):
    for i in range(n):
        o = env.reset()
        env.render()
        d = False
        while not d:
            board_state = np.array([
                o[0:3],
                o[3:6],
                o[6:]
            ])
            a = predict(board_state, 'saved-weights/-0.141.h5f')
            o, r, d, _ = env.step(a)
            env.render()
        if(r == 0):
            print("Deu velha")
        elif(r == 1):
            print("Bot ganhou")
        elif(r == -1):
            print("Bot Perdeu")
        print("-------------------")
            

In [45]:
playDql(2)

[[0 0 0]
 [1 0 0]
 [0 0 0]]
[[1 0 0]
 [1 2 0]
 [0 0 0]]
[[1 0 1]
 [1 2 0]
 [2 0 0]]
[[1 0 1]
 [1 2 0]
 [2 2 1]]
[[1 2 1]
 [1 2 0]
 [2 2 1]]
Bot ganhou
-------------------
[[1 0 0]
 [0 0 0]
 [0 0 0]]
[[1 0 0]
 [0 2 0]
 [0 1 0]]
[[1 0 0]
 [1 2 0]
 [2 1 0]]
[[1 0 2]
 [1 2 0]
 [2 1 0]]
Bot ganhou
-------------------


# PPO x DQN

O nosso teste final, é claro, colocar o nosso Agente PPO treinado para jogar contra o Agente do DQN. Façam suas apostas.

Para jogar, utilizaremos a função <code>stepX1</code>, feita no <code>TicTacToeEnv</code>. Esta função difere da <code>step</code> utilizada anteriormente, pois não faremos um movimento aleatório para o jogador 1, apenas colocará 1 ou 2, dependendo da vez de quem for, em uma posição do tabuleiro. 

Portanto, nossa função <code>stepX1</code> funciona da seguinte forma:

- Recebe uma posição da matriz que deve ser jogada;

- Transforma essa posição em uma coordenada relativa x,y;

- Escreve na matriz 1 ou 2 na posição recebida;

- Verifica se há um ganhador. A função de verificar um ganhador pode retornar 4 valores diferentes:
    - 0, se a partida ainda não acabou;
    - 1, se o jogador 1 ganhou;
    - 2, se o jogador 2 ganhou;
    - 3, se a partida empatou
    
- Por fim, a função retorna o tabuleiro atual após a jogada feita e o resultado da chamada da função winner.

Desta forma, faremos 2 testes diferentes: 

- Colocaremos, primeiro, o PPO como jogador número 1 e o DQN como jogador número 2.
- Depois, faremos o inverso, colocaremos o DQN como jogador número 1 e o PPO como jogador número 2.

Desta forma, veremos se, em algum dos agentes treinados, faz diferença ele começar jogando ou não. Esperamos que não ja que, no treinamento de ambos, os bots sempre jogavam em segundo.

Ainda, para jogar com o PPO, utilizamos a função <code>get_action</code> e, para jogar com o DQN utilizamos a função <code>predict</code>.

In [63]:

def ppoxdqn(train=False, visualize=False):
    turn = 1
    
    o = env.resetX1()
    if(visualize):
        env.render()
        print("PPO: 1")
        print("DQN: 2")
        print("-------------------")
        
    w = 0
    while w == 0:
        if(turn == 1):
            print("PPO turn")
            a = get_action(o)
            o, w = env.stepX1(a)
            turn = 2

        elif(turn == 2):
            print("DQN turn")
            board_state = np.array([
                o[0:3],
                o[3:6],
                o[6:]
            ])

            a = predict(board_state, 'saved-weights/-0.141.h5f')
            o, w = env.stepX1(a)
            turn = 1

        if(visualize):
            env.render()
            print("-------------------")
            
    if(visualize):
        if(w == 1):
            print("PPO ganhou")
        elif(w == 2):
            print("DQN ganhou")
        elif(w == 3):
            print("Deu velha")
        
        print("-------------------")
        
    return w
            

In [64]:
ppoxdqn(visualize=True)

[[0 0 0]
 [0 0 0]
 [0 0 0]]
PPO: 1
DQN: 2
-------------------
PPO turn
[[0 0 0]
 [0 0 0]
 [0 0 1]]
-------------------
DQN turn
[[0 0 0]
 [0 2 0]
 [0 0 1]]
-------------------
PPO turn
[[0 0 0]
 [0 2 0]
 [1 0 1]]
-------------------
DQN turn
[[0 0 0]
 [0 2 0]
 [1 2 1]]
-------------------
PPO turn
[[1 0 0]
 [0 2 0]
 [1 2 1]]
-------------------
DQN turn
[[1 2 0]
 [0 2 0]
 [1 2 1]]
-------------------
DQN ganhou
-------------------


2

In [20]:
winners = []
for i in range(50):
    w = ppoxdqn(train=True)
    winners.append(w)
    print(w, end = ' ')

3 3 2 3 1 3 3 3 3 1 

AttributeError: 'list' object has no attribute 'plot'

In [65]:
def dqnxppo(train=False, visualize=False):
    turn = 1
    
    o = env.resetX1()
    if(visualize):
        env.render()
        print("DQN: 1")
        print("PPO: 2")
        print("-------------------")
        
    w = 0
    while w == 0:
        if(turn == 2):
            print("PPO turn")
            a = get_action(o)
            o, w = env.stepX1(a)
            turn = 1

        elif(turn == 1):
            print("DQN turn")
            board_state = np.array([
                o[0:3],
                o[3:6],
                o[6:]
            ])

            a = predict(board_state, 'saved-weights/-0.141.h5f')
            o, w = env.stepX1(a)
            turn = 2

        if(visualize):
            env.render()
            print("-------------------")
            
    if(visualize):
        if(w == 1):
            print("DQN ganhou")
        elif(w == 2):
            print("PPO ganhou")
        elif(w == 3):
            print("Deu velha")
        
        print("-------------------")
        
    return w
            

In [66]:
dqnxppo(visualize=True)

[[0 0 0]
 [0 0 0]
 [0 0 0]]
DQN: 1
PPO: 2
-------------------
DQN turn
[[0 1 0]
 [0 0 0]
 [0 0 0]]
-------------------
PPO turn
[[0 1 0]
 [0 2 0]
 [0 0 0]]
-------------------
DQN turn
[[1 1 0]
 [0 2 0]
 [0 0 0]]
-------------------
PPO turn
[[1 1 0]
 [0 2 0]
 [2 0 0]]
-------------------
DQN turn
[[1 1 1]
 [0 2 0]
 [2 0 0]]
-------------------
DQN ganhou
-------------------


1

In [ ]:
winners = []
for i in range(50):
    w = dqnxppo(train=True)
    winners.append(w)
    print(w, end = ' ')

# Jogue contra os bots

In [ ]:
def humanxbot(bot, play_first):
    print(f"Bem vindo ao duelo, você duelará com {bot})
    if(play_first):
        print("Voce escolheu jogar primeiro")
    else:
        print("Voce escolheu jogar em segundo") 
          
    o = env.resetX1()
    

    if(visualize):
        env.render()
        print("DQN: 1")
        print("PPO: 2")
        print("-------------------")
        
    w = 0
    while w == 0:
        if(turn == 2):
            print("PPO turn")
            a = get_action(o)
            o, w = env.stepX1(a)
            turn = 1

        elif(turn == 1):
            print("DQN turn")
            board_state = np.array([
                o[0:3],
                o[3:6],
                o[6:]
            ])

            a = predict(board_state, 'saved-weights/-0.141.h5f')
            o, w = env.stepX1(a)
            turn = 2

        if(visualize):
            env.render()
            print("-------------------")
            
    if(visualize):
        if(w == 1):
            print("DQN ganhou")
        elif(w == 2):
            print("PPO ganhou")
        elif(w == 3):
            print("Deu velha")
        
        print("-------------------")
        
    return w
            